To-do:
- Run connectivity on 5 rest subjects (adjust parallelization as needed)
- Assess reliability (permutation tests if not obvious)
- Run averaging and SRM code
- Figure out task decoding and write task_decoding.py


To think about:
- Searchlights instead of parcels as connectivity targets, or as SRM boundaries
- Distance as SRM penalty instead of parcelwise
- DiFuMo atlas instead of Schaefer?
- Reliability.py: permutation tests?
- Write task_decoding.py

In [1]:
import glob
import numpy as np
import nibabel as nib
from nilearn import datasets
from nilearn.image import resample_img, math_img
from nilearn.maskers import MultiNiftiMasker, MultiNiftiLabelsMasker

In [1]:
from connectivity import load_data

In [2]:
files = ['data/rest/sub-s03_ses-01_task-rest_run-1_space-MNI_desc-optcomDenoised_bold.nii.gz']
confound_files = ['data/confounds/sub-s03_ses-01_task-rest_run-1_desc-confounds_timeseries.tsv']